In [1]:
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
import pickle

print(tf.__version__)

2.2.0


In [2]:
df = pd.read_csv ('Dataset1.csv')
print (df)

                                        INTENT  ID Unnamed: 2
0                       Undo the last sentence   1        NaN
1                           Undo the last word   1        NaN
2               Can you undo the last sentence   1        NaN
3                         Please undo the text   1        NaN
4                       Undo the selected text   1        NaN
..                                         ...  ..        ...
423             Please delete all text in bold  26        NaN
424             Kindly delete all text in bold  26        NaN
425                   Erase the last paragraph  26        NaN
426  Erase the last two words of that sentence  26        NaN
427                      Please erase all text  26        NaN

[428 rows x 3 columns]


In [3]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)

In [4]:
for index, row in df_shuffled.iterrows():

    tts_string=row['INTENT']
    tts_string=tts_string.lower() #lower case

    #lemmatize, then remove stopwords
    lemmatizer=WordNetLemmatizer()
    lemmatised_string=word_tokenize(tts_string)
    tts_string=""
    for word in lemmatised_string:
        tts_string=tts_string+lemmatizer.lemmatize(word)+" "
    tts_string=tts_string.strip()
    print(tts_string)

    #remove stopwords
    tokens = word_tokenize(tts_string)
    result = [i for i in tokens if not i in STOPWORDS]
    
    
    custom_stopwords = ['please','could','kindly']
    # using list comprehension to perform task 
    result = [i for i in result if i not in custom_stopwords] 


    print(result)
    
    preprocessed_str=""
#     print("here",preprocessed_str)
    for i in result:
        preprocessed_str=preprocessed_str+" "+i
#     preprocessed_str=' '.join(map(str, result))  
    print("Pre processed string",preprocessed_str)
    preprocessed_str=preprocessed_str.strip()
    # row['INTENT']=preprocessed_string

    #remove certain words
    #lemmatization
    #convert digits to numerals
    #text canonicalization?

    df_shuffled.at[index,'INTENT']=preprocessed_str

can you please clear italic
['clear', 'italic']
Pre processed string  clear italic
could you undo the selected text
['undo', 'selected', 'text']
Pre processed string  undo selected text
emphasize that for me
['emphasize']
Pre processed string  emphasize
can you remove underline for that word
['remove', 'underline', 'word']
Pre processed string  remove underline word
please show help
['show', 'help']
Pre processed string  show help
can you end bullet
['end', 'bullet']
Pre processed string  end bullet
please take a break from dictation
['take', 'break', 'dictation']
Pre processed string  take break dictation
unbold that for me
['unbold']
Pre processed string  unbold
remove all formatting from this page
['remove', 'formatting', 'page']
Pre processed string  remove formatting page
right align the first word of selected paragraph
['right', 'align', 'first', 'word', 'selected', 'paragraph']
Pre processed string  right align first word selected paragraph
italicize that last two sentence
['ita

Pre processed string  remove superscript
underline first n word in subscript
['underline', 'first', 'n', 'word', 'subscript']
Pre processed string  underline first n word subscript
please stop bullet
['stop', 'bullet']
Pre processed string  stop bullet
remove emphasis for the selected text
['remove', 'emphasis', 'selected', 'text']
Pre processed string  remove emphasis selected text
could you undo that for me
['undo']
Pre processed string  undo
select the text or number that you want to superscript
['select', 'text', 'number', 'want', 'superscript']
Pre processed string  select text number want superscript
centre align the last word of selected paragraph
['centre', 'align', 'last', 'word', 'selected', 'paragraph']
Pre processed string  centre align last word selected paragraph
remove bold for me
['remove', 'bold']
Pre processed string  remove bold
centre the content
['centre', 'content']
Pre processed string  centre content
underline that word please
['underline', 'word']
Pre processed

Pre processed string  show command
can you make it underline for me
['make', 'underline']
Pre processed string  make underline
can you show command
['show', 'command']
Pre processed string  show command
can you centre-align the line
['centre-align', 'line']
Pre processed string  centre-align line
please insert bullet
['insert', 'bullet']
Pre processed string  insert bullet
align the line to the left
['align', 'line', 'left']
Pre processed string  align line left
please pause dictation
['pause', 'dictation']
Pre processed string  pause dictation
can you please remove formatting from last 2 line
['remove', 'formatting', 'last', '2', 'line']
Pre processed string  remove formatting last 2 line
click the cursor to position to remove superscript
['click', 'cursor', 'position', 'remove', 'superscript']
Pre processed string  click cursor position remove superscript
can you strikethrough font size
['strikethrough', 'font', 'size']
Pre processed string  strikethrough font size
please start inser

In [5]:
vocab_size = 1000
embedding_dim = 20
max_length = 10
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = 0.8

In [6]:
train_size = int(428*training_portion)
X=np.array(df_shuffled['INTENT'].to_list())
Y_arr=np.array(df_shuffled['ID'].to_list())
Y=[]
for val in Y_arr:
    d=[val]
    Y.append(d)
    
train_X = X[0: train_size]
train_Y = Y[0: train_size]

validation_X = X[train_size:]
validation_Y = Y[train_size:]

print(train_size)
print(len(train_X))
print(len(train_Y))
print(train_Y)
print(len(validation_X))
print(len(validation_Y))

342
342
342
[[5], [1], [2], [7], [25], [21], [22], [3], [18], [17], [4], [2], [26], [13], [26], [16], [23], [17], [8], [18], [4], [7], [18], [1], [25], [5], [18], [12], [9], [23], [4], [23], [13], [7], [25], [4], [24], [5], [23], [17], [21], [22], [15], [14], [26], [8], [3], [15], [24], [3], [12], [13], [2], [20], [2], [6], [18], [2], [26], [14], [18], [23], [13], [3], [14], [25], [25], [16], [19], [19], [13], [6], [18], [18], [13], [1], [12], [1], [3], [11], [13], [5], [16], [8], [6], [25], [4], [2], [4], [13], [12], [10], [5], [17], [11], [14], [25], [18], [16], [13], [8], [10], [6], [24], [6], [15], [10], [10], [24], [1], [6], [24], [23], [1], [26], [14], [26], [15], [4], [3], [18], [18], [18], [13], [6], [13], [10], [21], [9], [26], [19], [25], [15], [20], [15], [23], [23], [12], [5], [7], [12], [6], [2], [1], [11], [20], [6], [4], [15], [6], [18], [1], [2], [2], [9], [10], [21], [3], [1], [8], [14], [3], [14], [6], [2], [23], [21], [13], [6], [18], [4], [1], [4], [23], [20], [19],

In [7]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_X)
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])

{'<OOV>': 1,
 'last': 2,
 'word': 3,
 'remove': 4,
 'text': 5,
 'selected': 6,
 'bold': 7,
 'underline': 8,
 'align': 9,
 'dictation': 10}

In [8]:
train_sequences = tokenizer.texts_to_sequences(train_X)
print(train_sequences[1])

[15, 6, 5]


In [9]:

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

2
10
3
10
4
10


In [10]:
validation_sequences = tokenizer.texts_to_sequences(validation_X)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)

86
(86, 10)


In [11]:
validation_Y=np.array(validation_Y)
training_Y=np.array(train_Y)
print(validation_Y.shape)
print(training_Y.shape)

(86, 1)
(342, 1)


In [12]:
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=10),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#     use ReLU in place of tanh function since they are very good alternatives of each other.
#     tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 27(26+0) units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(27, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 20)            20000     
_________________________________________________________________
bidirectional (Bidirectional (None, 40)                6560      
_________________________________________________________________
dense (Dense)                (None, 27)                1107      
Total params: 27,667
Trainable params: 27,667
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 60
history = model.fit(train_padded, training_Y, epochs=num_epochs, validation_data=(validation_padded, validation_Y), verbose=2)

Epoch 1/60
11/11 - 1s - loss: 3.2941 - accuracy: 0.0614 - val_loss: 3.2877 - val_accuracy: 0.1163
Epoch 2/60
11/11 - 0s - loss: 3.2825 - accuracy: 0.1462 - val_loss: 3.2788 - val_accuracy: 0.1163
Epoch 3/60
11/11 - 0s - loss: 3.2701 - accuracy: 0.1374 - val_loss: 3.2684 - val_accuracy: 0.1163
Epoch 4/60
11/11 - 0s - loss: 3.2527 - accuracy: 0.1462 - val_loss: 3.2544 - val_accuracy: 0.1279
Epoch 5/60
11/11 - 0s - loss: 3.2280 - accuracy: 0.1316 - val_loss: 3.2340 - val_accuracy: 0.1047
Epoch 6/60
11/11 - 0s - loss: 3.1927 - accuracy: 0.1082 - val_loss: 3.2094 - val_accuracy: 0.0698
Epoch 7/60
11/11 - 0s - loss: 3.1441 - accuracy: 0.1404 - val_loss: 3.1708 - val_accuracy: 0.1628
Epoch 8/60
11/11 - 0s - loss: 3.0700 - accuracy: 0.1930 - val_loss: 3.0731 - val_accuracy: 0.1628
Epoch 9/60
11/11 - 0s - loss: 2.9299 - accuracy: 0.2047 - val_loss: 2.9035 - val_accuracy: 0.2093
Epoch 10/60
11/11 - 0s - loss: 2.7697 - accuracy: 0.1988 - val_loss: 2.7708 - val_accuracy: 0.2442
Epoch 11/60
11/11 -

In [14]:
accr = model.evaluate(validation_padded,validation_Y)
# print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

3/3 [==============================] - 0s 2ms/step - loss: 0.2407 - accuracy: 0.9651


In [20]:
# new_commands = [['please stop my dictation here'],['subscript the last digit'],['superscript'],['remove italics here'],['end my bullet list'],['can you display all commands'],['help me'],['pause my dictation'],['please remove that bold'],['undo that'],['can you delete the last word'],['remove that strikethrough'],['centre align the content please'],['can you right align this image'],['could you remove formatting for the last word'],['insert a comment here'],['remove underline for last sentence'],['superscript that'],['can you remove subscript from here'],['Please go to next bullet'],['remove subscript'],['Start inserting bullet'],['please open commands'],['can you undo this'],['remove the underline here'],['align content left'],['please bold that'],['underline please'],['right align the text please'],['can you make it in italics'],['please make this italic'],['can you remove the subscript'],['move text right'],['strikethrough the last text'],['show help please'],['insert a bullet here']]
new_commands = [['hey underline the last word'],['remove italics'],['Stop my dictation'],['insert a bullet here'],['MOve content to right']]


In [21]:
new_commands_processed=[]
    
for i,ele in enumerate(new_commands):

    tts=ele[0]
    print(tts)
    tts=tts.lower() #lower case

    #lemmatize, then remove stopwords
    lemmatizer=WordNetLemmatizer()
    lemmatised_string=word_tokenize(tts)
    tts=""
    for word in lemmatised_string:
        tts=tts+lemmatizer.lemmatize(word)+" "
    tts=tts.strip()
    # print(tts_string)

    #remove stopwords
    tokens = word_tokenize(tts)
    result = [i for i in tokens if not i in STOPWORDS]
    
    
    custom_stopwords = ['please','could','kindly']
    # using list comprehension to perform task 
    result = [i for i in result if i not in custom_stopwords] 

    
    print(result)
    preprocessed_string=""
    for i in result:
        preprocessed_string=preprocessed_string+" "+i
#     preprocessed_string= ' '.join(map(str, result)) 
    print(preprocessed_string)
    preprocessed_string=preprocessed_string.strip()
    str0=[preprocessed_string]
#     d=[val]
    new_commands_processed.append(str0)
#     new_commands[i]=preprocessed_string
    # row['INTENT']=preprocessed_string

    #remove certain words
    #lemmatization
    #convert digits to numerals
    #text canonicalization?

    

# print(new_commands')
    

hey underline the last word
['hey', 'underline', 'last', 'word']
 hey underline last word
remove italics
['remove', 'italic']
 remove italic
Stop my dictation
['stop', 'dictation']
 stop dictation
insert a bullet here
['insert', 'bullet']
 insert bullet
MOve content to right
['move', 'content', 'right']
 move content right


In [22]:
label={'0':'','1':'UNDO','2':'BOLD','3':'REMOVE_BOLD','4':'ITALIC','5':'REMOVE_ITALIC','6':'UNDERLINE','7':'REMOVE_UNDERLINE','8':'SUPERSCRIPT','9':'REMOVE_BOLD','10':'SUBSCRIPT','11':'REMOVE_SUBSCRIPT','12':'STRIKETHROUGH','13':'REMOVE_STRIKETHROUGH','14':'ALIGN_CENTER','15':'INSERT_COMMENT','16':'ALIGN_LEFT','17':'ALIGN_RIGHT','18':'REMOVE_FORMATTING','19':'INSERT_BULLET','20':'NEXT_BULLET','21':'END_BULLET','22':'PAUSE_DICTATION','23':'STOP_DICTATION','24':'SHOW_ALL_COMMANDS','25':'SHOW_HELP','26':'DELETE'}
label_list=list_values = [ v for v in label.values() ]
# print(new_commands_processed)
for dum in new_commands_processed :
    print(dum)
    sequences = tokenizer.texts_to_sequences(dum)
#     print(sequences)
    padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    print(padded)
    # seq = tokenizer.texts_to_sequences()
    # padded = pad_sequences(seq, maxlen=max_length)
    # print(padded)
    pred = model.predict(padded)
    print(label_list[np.argmax(pred)],"\n")
#     print(np.argmax(pred),"\n")
#     print(pred)

['hey underline last word']
[[1 8 2 3 0 0 0 0 0 0]]
UNDERLINE 

['remove italic']
[[ 4 26  0  0  0  0  0  0  0  0]]
REMOVE_ITALIC 

['stop dictation']
[[23 10  0  0  0  0  0  0  0  0]]
STOP_DICTATION 

['insert bullet']
[[20 12  0  0  0  0  0  0  0  0]]
INSERT_BULLET 

['move content right']
[[56 58 24  0  0  0  0  0  0  0]]
ALIGN_RIGHT 



In [23]:
model.save('weights/lstm_3next_b')

INFO:tensorflow:Assets written to: weights/lstm_3next_b/assets


In [24]:

import tensorflow.keras as keras

tf.keras.models.save_model(model,"weights/3nextb")

INFO:tensorflow:Assets written to: weights/3nextb/assets


In [27]:

# Converting a SavedModel to a TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_saved_model("weights/3nextb")
tflite_model = converter.convert()


In [28]:
# Save the TF Lite model.
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)